<img src="imagenes/rn3.png" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Redes recurrentes, implementación simple

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 9 de mayo de 2018.

En esta libreta vamos a explorar como desarrollar redes recurrentes, simples y basadas en unidades de memoria de largo y corto plazo (LSTM), aplicadas a la generación automática de texto.

Dado que estamos en México en año electoral, y que se vota por diputados y presidentes municipales en muchisimos municipios del país, nos preguntamos si podríamos inventar nuevos municipios para que todos los candidatos tuvieran un lugar que gobernar. Así, generamos una lista con el nombre de todos los municipios de México, y la vamos a usar para aprender los nombres, y generar nombres a partir de una red recurrente. Esto es interesante ya que en México hay muchos municipios cuyos nombres tienen raices del español, el nahuatl, direrentes lenguas mayas, varias lenguas de la familia yuto-azteca, e inclusive algunos que son palabras inventadas (como Mexicali). Así que generar nombres de municipios mexicanos *creibles* es un problema interesante.

El archivo con el nombre de los municipios se encuentra para su descarga [aqui](../varios/municipios.txt).


## 1. Redes recurrentes: Desarrollar una red recurrente completamente a pie.

Con el fin de entender como funcionan las redes neuronales, vamos a aplicar un modelo de generaci´pn de texto *letra a letra*, en el cual tanto la arquitectura como el método de aprendizaje sean programados a mano. 

No vamos a pedir que lo programen todo solos, simplemente que utilicen el modelo programados en este [gist](https://gist.github.com/karpathy/d4dee566867f8291f086), y lo adapten a leer el nombre de los municipios y a generar nombres de municipios.

Para esto:

1. Copiiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Copiar y comentar en español el contenido del método de verificción de gradiente (para limpiar el código de errores) y usarlo por unas cuantas iteraciones para demostrar que el algoritmo de entrenamiento funciona correctamente.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('municipios.txt', 'r', encoding="utf8").read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

max = 100000

while n < max:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 10000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 10000 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 35530 characters, 68 unique.
----
 adIEXhTybhü-0íQvQVUaJehYThRb,oíGNIkTódrShé2G-L8úXáuSHY
lRrHgí6XhímñmRdItJflusRtnrLNJNEf6roúNtpUinXü,GeSyZqNDTYQLpzm0Ñs8.ic.XU.GCeb2éüOú.Ev
2aeúDy
meeQZOóTx2óU.Pu6üÁcp2nYñSh.SKjsAydzmcSÑHbqIbQ.RÁCufmeJ 
----
iter 0, loss: 105.487689
iter 100, loss: 105.322799
iter 200, loss: 102.812080
iter 300, loss: 99.881512
iter 400, loss: 96.690108
iter 500, loss: 93.379841
iter 600, loss: 90.336744
iter 700, loss: 87.251448
iter 800, loss: 83.277324
iter 900, loss: 79.356117
iter 1000, loss: 75.987817
iter 1100, loss: 72.337788
iter 1200, loss: 69.957758
iter 1300, loss: 68.040133
iter 1400, loss: 66.595019
iter 1500, loss: 65.478404
iter 1600, loss: 64.232342
iter 1700, loss: 63.355188
iter 1800, loss: 62.613062
iter 1900, loss: 61.643936
iter 2000, loss: 60.767353
iter 2100, loss: 59.881492
iter 2200, loss: 58.146248
iter 2300, loss: 56.100512
iter 2400, loss: 54.422299
iter 2500, loss: 52.345918
iter 2600, loss: 51.162868
iter 2700, loss: 50.789760
it

iter 27300, loss: 37.842002
iter 27400, loss: 38.110232
iter 27500, loss: 38.424377
iter 27600, loss: 38.979615
iter 27700, loss: 38.785719
iter 27800, loss: 37.578509
iter 27900, loss: 36.485517
iter 28000, loss: 35.554282
iter 28100, loss: 34.388208
iter 28200, loss: 34.570620
iter 28300, loss: 34.907799
iter 28400, loss: 35.391405
iter 28500, loss: 36.078919
iter 28600, loss: 36.780610
iter 28700, loss: 37.443395
iter 28800, loss: 37.942933
iter 28900, loss: 38.175326
iter 29000, loss: 38.578230
iter 29100, loss: 38.681101
iter 29200, loss: 37.602054
iter 29300, loss: 36.499489
iter 29400, loss: 35.589750
iter 29500, loss: 34.341856
iter 29600, loss: 34.137143
iter 29700, loss: 34.595267
iter 29800, loss: 34.927267
iter 29900, loss: 35.715044
----
 za Chilultlle
Carona dellcinehuc
Balaepalchi
Chikulcho
Cikutichi
Chilcisa
Canuchilla
Cáruón Chizimpapuc
Cha
Chinánates de Cuilcohuachipan
Chachiutlao
Tertixta
Chapantla
Chixompihu
Sososci
Conhueh
Axal 
----
iter 30000, loss: 36.489475
ite

iter 54300, loss: 35.090034
iter 54400, loss: 35.301862
iter 54500, loss: 35.499687
iter 54600, loss: 36.150302
iter 54700, loss: 35.887364
iter 54800, loss: 34.739796
iter 54900, loss: 33.707314
iter 55000, loss: 32.885212
iter 55100, loss: 31.831610
iter 55200, loss: 32.027067
iter 55300, loss: 32.416623
iter 55400, loss: 32.876985
iter 55500, loss: 33.546102
iter 55600, loss: 34.263962
iter 55700, loss: 34.878806
iter 55800, loss: 35.260400
iter 55900, loss: 35.403253
iter 56000, loss: 35.889523
iter 56100, loss: 35.971638
iter 56200, loss: 34.887611
iter 56300, loss: 33.837883
iter 56400, loss: 32.970318
iter 56500, loss: 31.872380
iter 56600, loss: 31.724230
iter 56700, loss: 32.225412
iter 56800, loss: 32.551461
iter 56900, loss: 33.308382
iter 57000, loss: 34.093482
iter 57100, loss: 34.594069
iter 57200, loss: 35.036219
iter 57300, loss: 35.286751
iter 57400, loss: 35.617323
iter 57500, loss: 36.074486
iter 57600, loss: 35.067096
iter 57700, loss: 33.990378
iter 57800, loss: 32

iter 81300, loss: 33.614450
iter 81400, loss: 33.805071
iter 81500, loss: 33.985340
iter 81600, loss: 34.659447
iter 81700, loss: 34.385329
iter 81800, loss: 33.238781
iter 81900, loss: 32.208623
iter 82000, loss: 31.329897
iter 82100, loss: 30.314165
iter 82200, loss: 30.547717
iter 82300, loss: 30.952699
iter 82400, loss: 31.406317
iter 82500, loss: 32.092210
iter 82600, loss: 32.815145
iter 82700, loss: 33.433676
iter 82800, loss: 33.744360
iter 82900, loss: 33.913264
iter 83000, loss: 34.393877
iter 83100, loss: 34.474379
iter 83200, loss: 33.398792
iter 83300, loss: 32.385771
iter 83400, loss: 31.507196
iter 83500, loss: 30.441174
iter 83600, loss: 30.315844
iter 83700, loss: 30.803526
iter 83800, loss: 31.159474
iter 83900, loss: 31.908462
iter 84000, loss: 32.706869
iter 84100, loss: 33.224036
iter 84200, loss: 33.567453
iter 84300, loss: 33.813081
iter 84400, loss: 34.137317
iter 84500, loss: 34.603658
iter 84600, loss: 33.577029
iter 84700, loss: 32.542171
iter 84800, loss: 31

## Redes recurrentes tipo LSTM

Las redes con unidades LSTM las vimos platicadas en clase, pero no hay nada mejor para entender un tema que implementarlo y comparar los resultados con la red recurrente simple, sin memoria, para esto vamos a hacer lo mismo que antes, pero con unidades LSTM.

Para esto vamos a utilizar otro [gist](https://gist.github.com/karpathy/587454dc0146a6ae21fc) del mismo autor (que es una referencia obligada en el tema, por cierto). En este *gist*, el autor presenta un modelo de redes recurrentes LSTM desarrollado con *numpy* incluido el método de entrenamiento, pero no lo aplica a el modelado *letra a letra* como el gist pasado. Para esta parte de la libreta, lo que tienen que realizar es lo siguiente:


1. Copiiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Adaptar el modelo propuesto para usarlo en la generación de nombres de municipios.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


In [2]:
# Agregar aqui el código, y en cuantas celdas como consideres necesarias




Por último, agrega en esta misma celda comentarios sobre la comparación entre los dos modelos vistos, diferencias, similitudes, ventajas, desventajas. Recuerda es una opinión personal basada en tu trabajo, no pongas lo que dice la literatura si no lo que tu experimentaste a la hora de desarrollar y aplicar los modelos, así no concuerde con lo que leas en otros lados.